In [2]:
using Revise

# A tutorial for the package ```ChebyshevFiltering```.

In this notebook we will provide a tutorial on how to use the the package ```ChebyshevFiltering``` for sparse diagonalization of Hermitian matrices (both real and complex).

Let us first load the package

In [3]:
using ChebyshevFiltering
include("/home/dario/Dropbox/delocalization_and_growth/revision_chaos_paper_code/QMB_module.jl")
using .ImpurityUtilities

and now let's create an SYK-like Hamiltonian. We also renormalize its spectrum to lie in the interval $\left[-1 , \, 1 \right]$.

In [4]:
const number_of_majorana = 20
majorana_matrices = majorana_matrices_chiral(number_of_majorana)
hamiltonian_matrix, edges_graph, weights_graph = hamiltonian_chiral(number_of_majorana, majorana_matrices, 4, 1.0)
renormalization_hamiltonian!(hamiltonian_matrix)

512×512 SparseArrays.SparseMatrixCSC{ComplexF64, Int64} with 14848 stored entries:
⎡⢿⣷⣯⣮⡿⠅⠀⢸⣾⡿⠇⠀⠀⠀⠀⠳⣄⠘⣶⡟⠳⣄⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠈⠳⣄⣝⣾⡮⎤
⎢⡫⣿⣿⣿⣼⣢⣶⣾⠛⠙⠂⠀⠀⠀⠀⢦⣼⠿⣏⠙⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⢐⢦⣮⡻⣗⠙⎥
⎢⠟⠏⠲⣻⣿⣿⣟⣦⡠⣤⣄⠀⢠⣀⣤⡾⠛⠆⠈⠳⠀⠀⠀⠈⠳⣄⠀⠀⠀⠠⣄⠀⢤⣨⡻⣿⠝⠆⠈⠳⎥
⎢⣀⣀⣸⣿⠻⣽⣿⣿⣾⢫⡉⢳⣤⡟⢧⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⢀⡈⣳⣷⣟⢯⠈⠳⣄⠀⠀⎥
⎢⣾⡿⣟⠀⠀⣮⡾⣛⣿⣿⣾⢿⡉⠓⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣔⣿⣿⡋⠳⣄⠀⠀⠈⠳⣄⎥
⎢⠉⠁⠈⠀⠀⠙⢧⣈⣾⣟⢿⣷⣯⣪⡿⠅⠀⢸⣾⡿⠳⣄⠀⠀⠈⠳⣄⢽⣮⡻⠳⣅⠁⠀⠈⠀⠀⠀⠀⠈⎥
⎢⠀⠀⠀⠀⠀⢲⣤⠿⢧⠈⡫⣻⣿⣿⣼⣦⣶⡞⠛⠙⠀⠈⠳⣄⢐⣦⡺⡿⣝⠙⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⎥
⎢⢤⡀⠠⣄⣠⡿⠉⠳⠀⠙⠟⠏⠲⣿⣿⣿⣏⡦⡠⣤⢄⠀⢠⡨⣷⣟⠟⠆⠈⠳⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⎥
⎢⣀⠙⣶⡟⠻⠄⠀⠀⠀⠀⣀⣀⣸⠿⠫⡽⣿⣿⣿⢯⡈⣱⣼⠿⠮⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⎥
⎢⣼⠿⣏⠙⢦⡀⠀⠀⠀⠀⣾⡿⣟⠀⠀⣮⡿⣟⢿⣷⣾⡿⡋⠱⢄⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⎥
⎢⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠑⢆⣨⣾⡿⢿⣷⣽⣾⡻⠀⠀⣽⣾⡿⠀⠀⠀⠀⠈⠳⣄⣹⣶⡟⎥
⎢⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⡲⣶⡟⢏⡈⣳⣿⣿⣿⣞⣢⣶⡏⠉⠉⠀⠀⠀⠀⠐⣦⣼⠿⣄⠉⎥
⎢⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⢦⡀⠰⣴⣽⢿⡊⠃⠀⠑⠛⠊⠺⣹⣿⣿⣿⠦⣰⣴⣄⠀⢦⣀⣾⠋⠙⠂⠈⠓⎥
⎢⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⣄⣝⣾⡮⠻⠅⠙⢦⡀⠀⣄⣤⡼⠿⠻⡟⣿⣿⣯⣪⡀⢳⣶⠛⠧⠀⠀⠀⠀⠀⎥
⎢⡀⠀⠀⠀⠀⡀⠀⢀⢙⢦⣮⡻⣗⠙⢦⡀⠀⠀⠙⢦⣾⡿⡇⠀⢐⣾⡫⣻⢿⣷⣽⡿⡉⢳⣄⠀⠀⡀⢀⣀⎥
⎢⠙⢦⡀⠀⠀⠙⢦⣨⣿⣿⠝⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢤⣈⣷⡿⣿⣿⣭⡾⡻⠀⠀⣽⣾⡿⎥
⎢⠀⠀⠙⢦⡀⣳⣽⢿⢯⡈⠁⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⢳⣼⠛⢧⣈⣣⡿⣿⣿⣟⣦⣿⡏⠉⠉⎥
⎢⢦⡀⠰⣔⣿⣮⡋⠓⠀⠙⠂⠀⠀⠀⠙⢦⡀⠀⠀⠀⢦⡀⠰⣤⡾⠛⠉⠃⠀⠙⠛⠊⠻⣽⣿⣿⣯⠦⣰⣴⎥
⎢⣄⢽⣮⡻⠳⠅⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⣄⣹⣶⡟⠳⠀⠀⠀⠀⠠⣄⣤⡿⠿⠫⡟⣿⣿⣿⣪⎥
⎣⡺⡿⣝⠙⢦⡀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠙⢦⣼⠿⡄⠙⢦⠀⠀⠀⠀⢰⣾⡿⡇⠀⢐⣾⡻⣻⢿⣷⎦

## The estimate of the density of states via Kernel Polynomial Method.

We now test how works the estimate of the density of states, computed by means of the Kernel Polynomial Method.

We start by fixing the relevant parameters, and the window of the spectrum where we want to estimate the density

In [5]:
max_degree_KPM = 150
stochastic_dimension_KPM = 30
lambda_min = -0.1
lambda_max = 0.1

"log_try.txt"

and now we can compute the approximated density and compute the number of expected states

In [6]:
approximated_density = ChebyshevFiltering.KPM_density(hamiltonian_matrix, max_degree_KPM, stochastic_dimension_KPM)
x_values = lambda_min:((lambda_max - lambda_min) / 200):lambda_max
y_values = [approximated_density(x) for x in x_values]

using NumericalIntegration
expected_value = NumericalIntegration.integrate(x_values, y_values)

98.26430615829746

which we compare against the actual value:

In [7]:
using LinearAlgebra
real_number = length([value  for value in eigvals(Matrix(hamiltonian_matrix)) if lambda_min < value < lambda_max])

98

We see that the agreement is excellent.

## The eigenvalues via ```eigen_cheb```

Now we can test the Chebyshev filtering main function: the computation of the eigenvalues in a given interval by means of ```eigen_cheb```

In [ ]:
log_path = "/home/dario/Dropbox/delocalization_and_growth_data/log_try/"
log_file_name = "log_try.txt"
lambda_min = -0.1
lambda_max = 0.1

In [ ]:
converged_target_values, converged_target_vectors = eigen_cheb(hamiltonian_matrix, log_path, log_file_name, lambda_min, lambda_max)

In [11]:
isapprox(converged_target_values, [value  for value in eigvals(Matrix(hamiltonian_matrix)) if lambda_min < value < lambda_max])

true